In [1]:
!git clone https://github.com/d-m-silva/SelectiveForgetting.git

/var/lib/oar/.batch_job_bashrc: line 5: /home/dudasilva/.bashrc: No such file or directory
fatal: destination path 'SelectiveForgetting' already exists and is not an empty directory.


In [2]:
%cd SelectiveForgetting

/home/dudasilva/SelectiveForgetting/SelectiveForgetting


/home/dudasilva/unlearning/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
import gc
 
# Returns the number of
# objects it has collected
# and deallocated
collected = gc.collect()

In [4]:
%load_ext autoreload

%autoreload 2

%matplotlib inline

import os

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'

os.environ['CUDA_VISIBLE_DEVICES']='0'

import variational

import matplotlib.pyplot as plt

import seaborn as sns

from matplotlib.ticker import FuncFormatter

import os

import time

import math

import pandas as pd

from collections import OrderedDict

from sklearn.linear_model import LogisticRegression



import copy

import torch.nn as nn

from torch.autograd import Variable

from typing import List

import itertools

from tqdm.autonotebook import tqdm

from models import *

import models

from logger import *

/tmp/ipykernel_13957/2905006116.py:45: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
def pdb():

    import pdb

    pdb.set_trace
    
def parameter_count(model):

    count=0

    for p in model.parameters():

        count+=np.prod(np.array(list(p.shape)))

    print(f'Total Number of Parameters: {count}')
    
def vectorize_params(model):

    param = []

    for p in model.parameters():

        param.append(p.data.view(-1).cpu().numpy())

    return np.concatenate(param)
    
def print_param_shape(model):

    for k,p in model.named_parameters():

        print(k,p.shape)

 ## Pre-training

In [6]:
#1) Pretraining on small_cifar10:

%run main.py --dataset small_cifar10 --model resnet --filters 0.4 --lr 0.1 --batch-size 128 --lossfn ce --weight-decay 0.0005 --seed 1

#2) Pretraining on cifar100:

#%run main.py --dataset cifar100 --model resnet --filters 0.4 --lr 0.1 --batch-size 128 --lossfn ce --weight-decay 0.0005 --seed 1

Checkpoint name: small_cifar10_resnet_0_4_forget_None_lr_0_1_bs_128_ls_ce_wd_0_0005_seed_1
[Logging in small_cifar10_resnet_0_4_forget_None_lr_0_1_bs_128_ls_ce_wd_0_0005_seed_1_training]


100%|██████████| 170M/170M [00:05<00:00, 29.5MB/s] 


Extracting /home/dudasilva/data/cifar-10-python.tar.gz to /home/dudasilva/data
Files already downloaded and verified
Number of Classes: 10
[0] train metrics:{"loss": 2.314283561706543, "error": 0.855}
Learning Rate : 0.1
[0] dry_run metrics:{"loss": 3.3192060661315916, "error": 0.86}
Learning Rate : 0.1
[0] test metrics:{"loss": 2.4529306049346924, "error": 0.8619999999850988}
Learning Rate : 0.1
Epoch Time: 3.92 sec
[1] train metrics:{"loss": 2.7997510623931885, "error": 0.88}
Learning Rate : 0.1
Epoch Time: 1.66 sec
[2] train metrics:{"loss": 3.0096083641052247, "error": 0.87}
Learning Rate : 0.1
Epoch Time: 1.92 sec
[3] train metrics:{"loss": 2.941478967666626, "error": 0.8625}
Learning Rate : 0.1
Epoch Time: 1.73 sec
[4] train metrics:{"loss": 2.389516658782959, "error": 0.8425}
Learning Rate : 0.1
Epoch Time: 1.77 sec
[5] train metrics:{"loss": 2.4274136066436767, "error": 0.795}
Learning Rate : 0.1
[5] dry_run metrics:{"loss": 2.2957512187957763, "error": 0.76}
Learning Rate : 0.

## Fine-tuning

In [8]:
#TRAINING ON small_cifar_5 !!! WITHOUT FORGETTING !!!

#1) Using pretained small_cifar10 as checkpoint -> lighter

%run main.py --dataset small_cifar5 --model resnet --filters 0.4 --lr 0.01 --resume checkpoints/small_cifar10_resnet_0_4_forget_None_lr_0_1_bs_128_ls_ce_wd_0_0005_seed_1_25.pt --disable-bn --weight-decay 0.1 --batch-size 128 --epochs 31 --seed 1

#2) Using pretained CIFAR100 as checkpoint -> heavier

#%run main.py --dataset small_cifar5 --model resnet --filters 0.4 --lr 0.01 --resume checkpoints/cifar100_resnet_0_4_forget_None_lr_0_1_bs_128_ls_ce_wd_0_0005_seed_1_25.pt --disable-bn --weight-decay 0.1 --batch-size 128 --epochs 31 --seed 1

Checkpoint name: small_cifar5_resnet_0_4_forget_None_lr_0_01_bs_128_ls_ce_wd_0_1_seed_1
[Logging in small_cifar5_resnet_0_4_forget_None_lr_0_01_bs_128_ls_ce_wd_0_1_seed_1_training]
Files already downloaded and verified
Files already downloaded and verified
Number of Classes: 5


/home/dudasilva/SelectiveForgetting/SelectiveForgetting/main.py:172: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(args.resume)


[0] train metrics:{"loss": 1.5159034852981568, "error": 0.6579999966621399}
Learning Rate : 0.01
[0] test metrics:{"loss": 1.4028399524688722, "error": 0.6360000001192093}
Learning Rate : 0.01
Epoch Time: 1.43 sec
[1] train metrics:{"loss": 1.333157444000244, "error": 0.5780000014305114}
Learning Rate : 0.01
Epoch Time: 0.9 sec
[2] train metrics:{"loss": 1.200620369911194, "error": 0.4879999980926514}
Learning Rate : 0.01
Epoch Time: 0.89 sec
[3] train metrics:{"loss": 1.1271758279800415, "error": 0.45599999856948853}
Learning Rate : 0.01
Epoch Time: 0.92 sec
[4] train metrics:{"loss": 1.1013252830505371, "error": 0.4479999933242798}
Learning Rate : 0.01
Epoch Time: 0.93 sec
[5] train metrics:{"loss": 1.0609872312545776, "error": 0.4379999980926514}
Learning Rate : 0.01
[5] test metrics:{"loss": 1.183127137184143, "error": 0.4920000014305115}
Learning Rate : 0.01
Epoch Time: 1.22 sec
[6] train metrics:{"loss": 1.0458361825942992, "error": 0.4400000052452087}
Learning Rate : 0.01
Epoch 

In [9]:
#TRAINING ON small_cifar_5 !!! WITH FORGETTING !!!

#1) Using pretained small_cifar10 as checkpoint -> lighter

%run main.py --dataset small_cifar5 --model resnet --filters 0.4 --lr 0.01 --resume checkpoints/small_cifar10_resnet_0_4_forget_None_lr_0_1_bs_128_ls_ce_wd_0_0005_seed_1_25.pt --disable-bn --weight-decay 0.1 --batch-size 128 --epochs 31 --forget-class 0 --num-to-forget 25 --seed 1

#2) Using pretained CIFAR100 as checkpoint -> heavier

#%run main.py --dataset small_cifar5 --model resnet --filters 0.4 --lr 0.01 --resume checkpoints/cifar100_resnet_0_4_forget_None_lr_0_1_bs_128_ls_ce_wd_0_0005_seed_1_25.pt --disable-bn --weight-decay 0.1 --batch-size 128 --epochs 31 --forget-class 0 --num-to-forget 25 --seed 1

Checkpoint name: small_cifar5_resnet_0_4_forget_0_num_25_lr_0_01_bs_128_ls_ce_wd_0_1_seed_1
[Logging in small_cifar5_resnet_0_4_forget_0_num_25_lr_0_01_bs_128_ls_ce_wd_0_1_seed_1_training]
Files already downloaded and verified
Files already downloaded and verified
Replacing indexes [26 86  2 55 75 93 16 73 54 95 53 92 78 13  7 30 22 24 33  8 43 62  3 71
 45]
Number of Classes: 5


/home/dudasilva/SelectiveForgetting/SelectiveForgetting/main.py:172: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(args.resume)


[0] train metrics:{"loss": 1.503102180480957, "error": 0.6419999990463257}
Learning Rate : 0.01
[0] test metrics:{"loss": 1.4044535818099975, "error": 0.6320000003576278}
Learning Rate : 0.01
Epoch Time: 1.28 sec
[1] train metrics:{"loss": 1.3137593870162965, "error": 0.56400000166893}
Learning Rate : 0.01
Epoch Time: 1.0 sec
[2] train metrics:{"loss": 1.1830432481765747, "error": 0.4959999957084656}
Learning Rate : 0.01
Epoch Time: 0.91 sec
[3] train metrics:{"loss": 1.1180360469818116, "error": 0.46200000095367433}
Learning Rate : 0.01
Epoch Time: 0.83 sec
[4] train metrics:{"loss": 1.0832451620101928, "error": 0.4479999933242798}
Learning Rate : 0.01
Epoch Time: 0.75 sec
[5] train metrics:{"loss": 1.0386791048049926, "error": 0.41999999380111697}
Learning Rate : 0.01
[5] test metrics:{"loss": 1.1925303316116334, "error": 0.5020000030994415}
Learning Rate : 0.01
Epoch Time: 1.09 sec
[6] train metrics:{"loss": 1.0323348574638367, "error": 0.4379999938011169}
Learning Rate : 0.01
Epoch

## Logs

In [10]:
log_dict={}

training_epochs=25

log_dict['epoch']=training_epochs

parameter_count(copy.deepcopy(model))

Total Number of Parameters: 1787741


## Loads checkpoints

In [13]:
import copy

model0 = copy.deepcopy(model)



arch = args.model

filters=args.filters

arch_filters = arch +'_'+ str(filters).replace('.','_')

augment = False

dataset = args.dataset

class_to_forget = args.forget_class

init_checkpoint = f"checkpoints/{args.name}_init.pt"

num_classes=args.num_classes

num_to_forget = args.num_to_forget

num_total = len(train_loader.dataset)

num_to_retain = num_total - num_to_forget

seed = args.seed

unfreeze_start = None



learningrate=f"lr_{str(args.lr).replace('.','_')}"

batch_size=f"_bs_{str(args.batch_size)}"

lossfn=f"_ls_{args.lossfn}"

wd=f"_wd_{str(args.weight_decay).replace('.','_')}"

seed_name=f"_seed_{args.seed}_"



num_tag = '' if num_to_forget is None else f'_num_{num_to_forget}'

unfreeze_tag = '_' if unfreeze_start is None else f'_unfreeze_from_{unfreeze_start}_'

augment_tag = '' if not augment else f'augment_'



m_name = f'checkpoints/{dataset}_{arch_filters}_forget_None{unfreeze_tag}{augment_tag}{learningrate}{batch_size}{lossfn}{wd}{seed_name}{training_epochs}.pt'

m0_name = f'checkpoints/{dataset}_{arch_filters}_forget_{class_to_forget}{num_tag}{unfreeze_tag}{augment_tag}{learningrate}{batch_size}{lossfn}{wd}{seed_name}{training_epochs}.pt'



model.load_state_dict(torch.load(m_name))

model0.load_state_dict(torch.load(m0_name))



#model.cuda()

#model0.cuda()



for p in model.parameters():

    p.data0 = p.data.clone()

for p in model0.parameters():

    p.data0 = p.data.clone()

/tmp/ipykernel_13957/286994155.py:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(m_name))
/tmp/ipykernel_13957/286994155.py:63: FutureWar

In [14]:
log_dict['args']=args

### Distance between w(D) and w(D_r)

In [16]:
def distance(model,model0):

    distance=0

    normalization=0

    for (k, p), (k0, p0) in zip(model.named_parameters(), model0.named_parameters()):

        space='  ' if 'bias' in k else ''

        current_dist=(p.data0-p0.data0).pow(2).sum().item()

        current_norm=p.data0.pow(2).sum().item()

        distance+=current_dist

        normalization+=current_norm

    print(f'Distance: {np.sqrt(distance)}')

    print(f'Normalized Distance: {1.0*np.sqrt(distance/normalization)}')

    return 1.0*np.sqrt(distance/normalization)

In [17]:
log_dict['dist_Original_Retrain']=distance(model,model0)

Distance: 0.2779924594026405
Normalized Distance: 0.005568791383362767


### Distance of w(D) from initialization

In [18]:
def ntk_init(resume,seed=1):

    manual_seed(seed)

    model_init = models.get_model(arch, num_classes=num_classes, filters_percentage=filters).to(args.device)

    model_init.load_state_dict(torch.load(resume))

    return model_init

In [19]:
model_init = ntk_init(init_checkpoint,args.seed)

for p in model_init.parameters():

    p.data0 = p.data.clone()

/tmp/ipykernel_13957/3555119760.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_init.load_state_dict(torch.load(resume))


In [20]:
log_dict['dist_Original_Original_init']=distance(model_init,model)

Distance: 1.3335334427337733
Normalized Distance: 0.02675236830634745


**Data loader**

In [21]:
train_loader_full, valid_loader_full, test_loader_full = datasets.get_loaders(dataset, batch_size=args.batch_size, seed=seed, augment=False, shuffle=True)

marked_loader, _, _ = datasets.get_loaders(dataset, class_to_replace=class_to_forget, num_indexes_to_replace=num_to_forget, only_mark=True, batch_size=1, seed=seed, augment=False, shuffle=True)



def replace_loader_dataset(data_loader, dataset, batch_size=args.batch_size, seed=1, shuffle=True):

    manual_seed(seed)

    loader_args = {'num_workers': 0, 'pin_memory': False}

    def _init_fn(worker_id):

        np.random.seed(int(seed))

    return torch.utils.data.DataLoader(dataset, batch_size=batch_size,num_workers=0,pin_memory=True,shuffle=shuffle)



forget_dataset = copy.deepcopy(marked_loader.dataset)

marked = forget_dataset.targets < 0

forget_dataset.data = forget_dataset.data[marked]

forget_dataset.targets = - forget_dataset.targets[marked] - 1

forget_loader = replace_loader_dataset(train_loader_full, forget_dataset, seed=seed, shuffle=True)



retain_dataset = copy.deepcopy(marked_loader.dataset)

marked = retain_dataset.targets >= 0

retain_dataset.data = retain_dataset.data[marked]

retain_dataset.targets = retain_dataset.targets[marked]

retain_loader = replace_loader_dataset(train_loader_full, retain_dataset, seed=seed, shuffle=True)



assert(len(forget_dataset) + len(retain_dataset) == len(train_loader_full.dataset))

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Replacing indexes [26 86  2 55 75 93 16 73 54 95 53 92 78 13  7 30 22 24 33  8 43 62  3 71
 45]


In [22]:
#Remove the checkpoints to clear some memory except init_checkpoint (used after!!!!)

checkpoints_folder = os.path.dirname(init_checkpoint)

files = os.listdir(checkpoints_folder)

for file in files:
    full_path = os.path.join(checkpoints_folder, file)
    if full_path != init_checkpoint:
        os.remove(full_path)

# NTK based Forgetting

**NTK update**

In [23]:
def delta_w_utils(model_init,dataloader,name='complete'):

    model_init.eval()

    dataloader = torch.utils.data.DataLoader(dataloader.dataset, batch_size=1, shuffle=False)

    G_list = []

    f0_minus_y = []

    for idx, batch in enumerate(dataloader):#(tqdm(dataloader,leave=False)):

        batch = [tensor.to(next(model_init.parameters()).device) for tensor in batch]

        input, target = batch

        if 'mnist' in args.dataset:

            input = input.view(input.shape[0],-1)

        target = target.cpu().detach().numpy()

        output = model_init(input)

        G_sample=[]

        for cls in range(num_classes):

            grads = torch.autograd.grad(output[0,cls],model_init.parameters(),retain_graph=True)

            grads = np.concatenate([g.view(-1).cpu().numpy() for g in grads])

            G_sample.append(grads)

            G_list.append(grads)

        if args.lossfn=='mse':

            p = output.cpu().detach().numpy().transpose()

            #loss_hess = np.eye(len(p))

            target = 2*target-1

            f0_y_update = p-target

        elif args.lossfn=='ce':

            p = torch.nn.functional.softmax(output,dim=1).cpu().detach().numpy().transpose()

            p[target]-=1

            f0_y_update = copy.deepcopy(p)

        f0_minus_y.append(f0_y_update)

    return np.stack(G_list).transpose(),np.vstack(f0_minus_y)

#### Jacobians and Hessians

In [24]:
model_init = ntk_init(init_checkpoint,args.seed)

G_r,f0_minus_y_r = delta_w_utils(copy.deepcopy(model),retain_loader,'complete') 

#Code expects a directory to store the NTK results, "NTK_data"

output_dir = "NTK_data"
os.makedirs(output_dir, exist_ok=True)

# Saves results in the created directory

np.save(os.path.join(output_dir, 'G_r.npy'), G_r)
np.save(os.path.join(output_dir, 'f0_minus_y_r.npy'), f0_minus_y_r)


#np.save('NTK_data/G_r.npy',G_r)

#np.save('NTK_data/f0_minus_y_r.npy',f0_minus_y_r)

del G_r, f0_minus_y_r



#model_init = ntk_init(init_checkpoint,args.seed)

G_f,f0_minus_y_f = delta_w_utils(copy.deepcopy(model),forget_loader,'retain') 


np.save('NTK_data/G_f.npy',G_f)

np.save('NTK_data/f0_minus_y_f.npy',f0_minus_y_f)

del G_f, f0_minus_y_f

# Cleanup
torch.cuda.empty_cache()
gc.collect()

/tmp/ipykernel_13957/3555119760.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_init.load_state_dict(torch.load(resume))


470

In [25]:
G_r = np.load('NTK_data/G_r.npy')

G_f = np.load('NTK_data/G_f.npy')

G = np.concatenate([G_r,G_f],axis=1)



np.save('NTK_data/G.npy',G)

del G, G_f, G_r



f0_minus_y_r = np.load('NTK_data/f0_minus_y_r.npy')

f0_minus_y_f = np.load('NTK_data/f0_minus_y_f.npy')

f0_minus_y = np.concatenate([f0_minus_y_r,f0_minus_y_f])



np.save('NTK_data/f0_minus_y.npy',f0_minus_y)

del f0_minus_y, f0_minus_y_r, f0_minus_y_f

# Cleanup
torch.cuda.empty_cache()
gc.collect()

62

This only requires access to the gradients and the initialization

### w_lin(D)

In [26]:
G = np.load('NTK_data/G.npy')
theta = G.transpose().dot(G) + num_total*args.weight_decay*np.eye(G.shape[1])
del G

theta_inv = np.linalg.inv(theta)

np.save('NTK_data/theta.npy',theta)
del theta

G = np.load('NTK_data/G.npy')
f0_minus_y = np.load('NTK_data/f0_minus_y.npy')
w_complete = -G.dot(theta_inv.dot(f0_minus_y))

np.save('NTK_data/theta_inv.npy',theta_inv)
np.save('NTK_data/w_complete.npy',w_complete)
del G, f0_minus_y, theta_inv, w_complete

### w_lin(D_r)

In [28]:
G_r = np.load('NTK_data/G_r.npy')
theta_r = G_r.transpose().dot(G_r) + num_to_retain*args.weight_decay*np.eye(G_r.shape[1])
del G_r

theta_r_inv = np.linalg.inv(theta_r)
np.save('NTK_data/theta_r.npy',theta_r)
del theta_r

G_r = np.load('NTK_data/G_r.npy')
f0_minus_y_r = np.load('NTK_data/f0_minus_y_r.npy')
w_retain = -G_r.dot(theta_r_inv.dot(f0_minus_y_r))

np.save('NTK_data/theta_r_inv.npy',theta_r_inv)
np.save('NTK_data/w_retain.npy',w_retain)
del G_r, f0_minus_y_r, theta_r_inv, w_retain

#### Scrubbing Direction

In [30]:
w_complete = np.load('NTK_data/w_complete.npy')
w_retain = np.load('NTK_data/w_retain.npy')
delta_w = (w_retain-w_complete).squeeze()

In [31]:
delta_w_copy = copy.deepcopy(delta_w)

#### Actual Change in Weights

In [33]:
delta_w_actual = vectorize_params(model0)-vectorize_params(model)

print(f'Actual Norm-: {np.linalg.norm(delta_w_actual)}')

print(f'Predtn Norm-: {np.linalg.norm(delta_w)}')

scale_ratio = np.linalg.norm(delta_w_actual)/np.linalg.norm(delta_w)

print('Actual Scale: {}'.format(scale_ratio))

log_dict['actual_scale_ratio']=scale_ratio

Actual Norm-: 0.2779897153377533
Predtn Norm-: 0.13829437972073208
Actual Scale: 2.0101302446210627


#### Trapezium trick

In [34]:
m_pred_error = vectorize_params(model)-vectorize_params(model_init)-w_retain.squeeze()
print(f"Delta w -------: {np.linalg.norm(delta_w)}")

inner = np.inner(delta_w/np.linalg.norm(delta_w),m_pred_error/np.linalg.norm(m_pred_error))
print(f"Inner Product--: {inner}")

if inner<0:
    angle = np.arccos(inner)-np.pi/2
    print(f"Angle----------:  {angle}")

    predicted_norm=np.linalg.norm(delta_w) + 2*np.sin(angle)*np.linalg.norm(m_pred_error)
    print(f"Pred Act Norm--:  {predicted_norm}")
else:
    angle = np.arccos(inner)
    print(f"Angle----------:  {angle}")

    predicted_norm=np.linalg.norm(delta_w) + 2*np.cos(angle)*np.linalg.norm(m_pred_error)
    print(f"Pred Act Norm--:  {predicted_norm}")

predicted_scale=predicted_norm/np.linalg.norm(delta_w)
predicted_scale
print(f"Predicted Scale:  {predicted_scale}")
log_dict['predicted_scale_ratio']=predicted_scale

Delta w -------: 0.13829437972073208
Inner Product--: -0.11583815160652983
Angle----------:  0.11609879044957316
Pred Act Norm--:  0.4260177139710507
Predicted Scale:  3.080513574241696


#### Normalized Inner Product between Prediction and Actual Scrubbing Update

In [35]:
def NIP(v1,v2):

    nip = (np.inner(v1/np.linalg.norm(v1),v2/np.linalg.norm(v2)))

    print(nip)

    return nip

In [36]:
nip=NIP(delta_w_actual,delta_w)

log_dict['nip']=nip

0.37400860388768165


#### Reshape delta_w

In [37]:
def get_delta_w_dict(delta_w,model):

    # Give normalized delta_w

    delta_w_dict = OrderedDict()

    params_visited = 0

    for k,p in model.named_parameters():

        num_params = np.prod(list(p.shape))

        update_params = delta_w[params_visited:params_visited+num_params]

        delta_w_dict[k] = torch.Tensor(update_params).view_as(p)

        params_visited+=num_params

    return delta_w_dict

### Metrics

In [38]:
from utils import *

def get_metrics(model,dataloader,criterion,samples_correctness=False,use_bn=False,delta_w=None,scrub_act=False):

    activations=[]

    predictions=[]

    if use_bn:

        model.train()

        dataloader = torch.utils.data.DataLoader(retain_loader.dataset, batch_size=128, shuffle=True)

        for i in range(10):

            for batch_idx, (data, target) in enumerate(dataloader):

                data, target = data.to(args.device), target.to(args.device)

                output = model(data)

    dataloader = torch.utils.data.DataLoader(dataloader.dataset, batch_size=1, shuffle=False)

    model.eval()

    metrics = AverageMeter()

    mult = 0.5 if args.lossfn=='mse' else 1

    for batch_idx, (data, target) in enumerate(dataloader):

        data, target = data.to(args.device), target.to(args.device)

        if args.lossfn=='mse':

            target=(2*target-1)

            target = target.type(torch.cuda.FloatTensor).unsqueeze(1)

        if 'mnist' in args.dataset:

            data=data.view(data.shape[0],-1)

        output = model(data)

        loss = mult*criterion(output, target)

        if samples_correctness:

            activations.append(torch.nn.functional.softmax(output,dim=1).cpu().detach().numpy().squeeze())

            predictions.append(get_error(output,target))

        metrics.update(n=data.size(0), loss=loss.item(), error=get_error(output, target))

    if samples_correctness:

        return metrics.avg,np.stack(activations),np.array(predictions)

    else:

        return metrics.avg

In [39]:
def activations_predictions(model,dataloader,name):

    criterion = torch.nn.CrossEntropyLoss()

    metrics,activations,predictions=get_metrics(model,dataloader,criterion,True)

    print(f"{name} -> Loss:{np.round(metrics['loss'],3)}, Error:{metrics['error']}")

    log_dict[f"{name}_loss"]=metrics['loss']

    log_dict[f"{name}_error"]=metrics['error']

    return activations,predictions

In [40]:
def predictions_distance(l1,l2,name):

    dist = np.sum(np.abs(l1-l2))

    print(f"Predictions Distance {name} -> {dist}")

    log_dict[f"{name}_predictions"]=dist

In [41]:
def activations_distance(a1,a2,name):

    dist = np.linalg.norm(a1-a2,ord=1,axis=1).mean()

    print(f"Activations Distance {name} -> {dist}")

    log_dict[f"{name}_activations"]=dist

### Original Model

In [43]:
m_D_r_activations,m_D_r_predictions=activations_predictions(model,retain_loader,'Original_Model_D_r')

m_D_f_activations,m_D_f_predictions=activations_predictions(model,forget_loader,'Original_Model_D_f')

m_D_t_activations,m_D_t_predictions=activations_predictions(model,test_loader_full,'Original_Model_D_t')

Original_Model_D_r -> Loss:0.823, Error:0.33473684210526317
Original_Model_D_f -> Loss:0.626, Error:0.12
Original_Model_D_t -> Loss:1.074, Error:0.43


### Retrain Model

In [44]:
m0_D_r_activations,m0_D_r_predictions=activations_predictions(model0,retain_loader,'Retrain_Model_D_r')

m0_D_f_activations,m0_D_f_predictions=activations_predictions(model0,forget_loader,'Retrain_Model_D_f')

m0_D_t_activations,m0_D_t_predictions=activations_predictions(model0,test_loader_full,'Retrain_Model_D_t')

Retrain_Model_D_r -> Loss:0.814, Error:0.32842105263157895
Retrain_Model_D_f -> Loss:0.993, Error:0.44
Retrain_Model_D_t -> Loss:1.075, Error:0.44


# Scrub using NTK

In [45]:
scale=predicted_scale

direction = get_delta_w_dict(delta_w,model)



model_scrub = copy.deepcopy(model)

for k,p in model_scrub.named_parameters():

    p.data += (direction[k]*scale).to(args.device)

In [46]:
ntk_D_r_activations,ntk_D_r_predictions=activations_predictions(model_scrub,retain_loader,'NTK_D_r')

ntk_D_f_activations,ntk_D_f_predictions=activations_predictions(model_scrub,forget_loader,'NTK_D_f')

ntk_D_t_activations,ntk_D_t_predictions=activations_predictions(model_scrub,test_loader_full,'NTK_D_t')

NTK_D_r -> Loss:0.815, Error:0.33052631578947367
NTK_D_f -> Loss:0.971, Error:0.44
NTK_D_t -> Loss:1.079, Error:0.444


In [47]:
predictions_distance(m0_D_f_predictions,m_D_f_predictions,'Retrain_Original_D_f')

predictions_distance(m0_D_f_predictions,ntk_D_f_predictions,'Retrain_NTK_D_f')

Predictions Distance Retrain_Original_D_f -> 8.0
Predictions Distance Retrain_NTK_D_f -> 0.0


### Activations Distance

In [48]:
activations_distance(m0_D_f_activations,m_D_f_activations,'Retrain_Original_D_f')

activations_distance(m0_D_r_activations,m_D_r_activations,'Retrain_Original_D_r')

activations_distance(m0_D_t_activations,m_D_t_activations,'Retrain_Original_D_t')

activations_distance(m0_D_f_activations,ntk_D_f_activations,'Retrain_NTK_D_f')

activations_distance(m0_D_r_activations,ntk_D_r_activations,'Retrain_NTK_D_r')

activations_distance(m0_D_t_activations,ntk_D_t_activations,'Retrain_NTK_D_t')

Activations Distance Retrain_Original_D_f -> 0.28995272517204285
Activations Distance Retrain_Original_D_r -> 0.11064248532056808
Activations Distance Retrain_Original_D_t -> 0.11899513006210327
Activations Distance Retrain_NTK_D_f -> 0.09318870306015015
Activations Distance Retrain_NTK_D_r -> 0.0781976580619812
Activations Distance Retrain_NTK_D_t -> 0.08619366586208344


# Fisher Forgetting

### Finetune and Fisher Helper

In [50]:
from utils import *

def get_metrics(model,dataloader,criterion,samples_correctness=False,use_bn=False,delta_w=None,scrub_act=False):

    activations=[]

    predictions=[]

    if use_bn:

        model.train()

        dataloader = torch.utils.data.DataLoader(retain_loader.dataset, batch_size=128, shuffle=True)

        for i in range(10):

            for batch_idx, (data, target) in enumerate(dataloader):

                data, target = data.to(args.device), target.to(args.device)

                output = model(data)

    dataloader = torch.utils.data.DataLoader(dataloader.dataset, batch_size=1, shuffle=False)

    model.eval()

    metrics = AverageMeter()

    mult = 0.5 if args.lossfn=='mse' else 1

    for batch_idx, (data, target) in enumerate(dataloader):

        data, target = data.to(args.device), target.to(args.device)

        if args.lossfn=='mse':

            target=(2*target-1)

            target = target.type(torch.cuda.FloatTensor).unsqueeze(1)

        if 'mnist' in args.dataset:

            data=data.view(data.shape[0],-1)

        output = model(data)

        if scrub_act:

            G = []

            for cls in range(num_classes):

                grads = torch.autograd.grad(output[0,cls],model.parameters(),retain_graph=True)

                grads = torch.cat([g.view(-1) for g in grads])

                G.append(grads)

            grads = torch.autograd.grad(output_sf[0,cls],model_scrubf.parameters(),retain_graph=False)

            G = torch.stack(G).pow(2)

            delta_f = torch.matmul(G,delta_w)

            output += delta_f.sqrt()*torch.empty_like(delta_f).normal_()



        loss = mult*criterion(output, target)

        if samples_correctness:

            activations.append(torch.nn.functional.softmax(output,dim=1).cpu().detach().numpy().squeeze())

            predictions.append(get_error(output,target))

        metrics.update(n=data.size(0), loss=loss.item(), error=get_error(output, target))

    if samples_correctness:

        return metrics.avg,np.stack(activations),np.array(predictions)

    else:

        return metrics.avg